## Entrenamiento

En este apartado vamos a usar cuatro modelos distintos con la finalidad
de comparar y luego encontrar el mejor modelo para nuestro dataset.

Para empezar vamos a cargar la data previamente procesada en el anterior apartado, de ese paso se obtuvo otro subconjunto de datos más pequeño con el cual podremos trabajar los modelos.

In [21]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

from xgboost import XGBRegressor
%matplotlib inline

In [17]:
df_taxi = pd.read_csv('train_cleared.csv')

In [18]:
df_taxi

,fare_amount,passenger_count,distance,day_status,time_status
0,7.7,1,2.802346,0,0
1,7.5,1,1.557516,1,3
2,16.5,1,4.160010,1,2
3,9.0,1,1.254608,1,2
4,5.3,1,1.376087,0,1
...,...,...,...,...,...
8487132,5.5,3,0.872517,1,1
8487133,5.7,3,1.664409,1,1
8487134,13.7,1,2.528474,1,3
8487135,4.5,1,1.163118,0,3


In [19]:
df_taxi['fare_amount'].describe()

count    8.487137e+06
mean     1.114520e+01
std      9.191402e+00
min      2.500000e+00
25%      6.000000e+00
50%      8.500000e+00
75%      1.250000e+01
max      4.750000e+02
Name: fare_amount, dtype: float64

In [26]:
# separamos las características que usaremos para predecir
# del resultado, en este caso el resultado que queremos obtener
# es 'fare_amount' a partir del resto de columnas
X = df_taxi.drop(['fare_amount'], axis=1)
y = df_taxi['fare_amount']

In [27]:
# separamos la data de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [28]:
print("Número de entradas de entrenamiento:", X_train.shape[0])
print("Número de entradas de prueba:", X_test.shape[0])

Número de entradas de entrenamiento: 5940995
Número de entradas de prueba: 2546142


In [34]:
## Usaremos una funcion para extraer las metricas del modelo
def model_metric(model, X_test, y_test, name_model):
    y_predict = model.predict(X_test)
    model_mse = np.sqrt(mean_squared_error(y_test, y_predict))
    model_r2 = r2_score(y_test, y_predict)
    print(f"El error cuadrático medio de {name_model} es:", model_mse)
    print(f"La métrica r2 score de {name_model} es:", model_r2)

In [31]:
# El primer modelo que probaremos será AdaBoost
ada = AdaBoostRegressor()

In [32]:
# Entrenamos el modelo
ada.fit(X_train, y_train)

AdaBoostRegressor()

In [35]:
model_metric(ada, X_test, y_test,'AdaBoostRegressor')

El error cuadrático medio de AdaBoostRegressor es: 6.008459980478541
La métrica r2 score de AdaBoostRegressor es: 0.5725619501856332
